In [518]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [519]:
pwd


u'/Users/camilogarciatorres/Documents/GitHub/Taller1'

In [520]:
base_dir = os.path.join( '.','data')
ciudad = 'Medellin'
fuente = 'NREL'
union = os.path.join(base_dir,ciudad,fuente+'.csv')
serieREL= pd.read_csv(union, sep = ';',
                encoding='latin-1')

base_dir2 = os.path.join( '.','data')
ciudad2 = 'Medellin'
fuente2 = 'IDEAM'
union2 = os.path.join(base_dir,ciudad,fuente+'.csv')
serieIDEAM= pd.read_csv(union, sep = ';',
                encoding='latin-1')



In [521]:
def MCP(mediax, mediay, sigmax, sigmay, x):
    return ((mediay-(sigmay/sigmax)*mediax)+(sigmay/sigmax)*x)

In [522]:
serieIDEAM.head()

,AÑO,MES,DIA,HORA,NREL
0,2005,1,1,0,0.0
1,2005,1,1,1,0.0
2,2005,1,1,2,0.0
3,2005,1,1,3,0.0
4,2005,1,1,4,0.0


In [523]:
mediay=serieREL['NREL'].mean()
sigmay=serieREL['NREL'].std()

In [524]:
for year in (2005,2006):
     for mes in (1,13):
            for dia in(1,31):
                dia=serieIDEAM[1:dia]
    
                                       

In [525]:
def listaobjetivo(ano1,ano2,mes1,mes2,dia1,dia2):
    for year in (ano1,ano2):
         for mes in (mes1,mes2):
            for dia in(dia1,dia2):
                
                numfinal=serieIDEAM.loc[((serieIDEAM["AÑO"] == year)
                            & (serieIDEAM["MES"] == mes)&  ,"DIAS"].item())
                lista=serieIDEAM[numinicial:numfinal]
                                        
return lista


SyntaxError: invalid syntax (<ipython-input-525-4a43a3b8983d>, line 7)

In [ ]:
dia

In [ ]:
hola=listaobjetivo(2006,2007,1,13,1,31)
hola


In [ ]:
listafinal